In [64]:
import os
from openai import OpenAI
import openai
from PIL import Image
import base64

In [65]:
os.environ["OPENAI_API_KEY"] = "sk-6OyTxsQcuf5tBz4w221b5822EfA143539f03A7F7BaB778D4"
os.environ["OPENAI_BASE_URL"] = "https://api.xiaoai.plus/v1"

client = OpenAI()

GPT_MODEL = "gpt-4o"

prompt = "give the brief description of image"
# prompt = "give the description of what to imaging in a line"
# prompt = "Generates prompt that is inspirational and suggestive, it can be used to recreate an uploaded image exactly"
# prompt = "Generates prompt briefly that is inspirational and suggestive, it can be used to recreate an uploaded image exactly"
# prompt = "Carefully analyze the provided image and generate an exhaustive description that captures every element essential for its recreation. Detail the spatial arrangement, key subjects, and any background elements. Specify the color palette, noting any gradients or unique shades present. Describe any actions or interactions between subjects, and capture the mood or atmosphere effectively. Your description should be precise and comprehensive, enabling an artist or a generative model to replicate the image with high fidelity based on your text alone."

In [66]:
# Encode the image at the given path to a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Get the base64 encoded description of an image at the given path
def get_image_description(image_path):
    encoded_image = encode_image(image_path)
    return encoded_image

# Generate a detailed description of the image using GPT-4
def generate_description_with_gpt(base64_image, prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    },
                ]
            }
        ],
        stream=False,
        max_tokens=300,
    )
    summary = response.choices[0].message.content.strip()
    return summary 

# Process a list of image paths and generate descriptions for each
def process_images(image_paths):
    descriptions = []
    for image_path in image_paths:
        image_description = get_image_description(image_path)
        description = generate_description_with_gpt(image_description, prompt)
        descriptions.append(description)
    return descriptions

In [67]:
def get_images_path(set_path):

    file_paths = []

    for root, _, files in os.walk(set_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    return file_paths

def write_prompt_out(descriptions):
    with open("output.txt", "w") as file:
        for line in descriptions:
            file.write(line + "\n")


In [68]:
if __name__ == "__main__":
    set_path = "./miniset"
    image_paths = get_images_path(set_path)
    descriptions = process_images(image_paths)
    write_prompt_out(descriptions)
    for i, description in enumerate(descriptions):
        print(f"Description for image {i+1}: {description}")

Description for image 1: A computer monitor with a padlock and chain securing its screen.
Description for image 2: A plate with sliced fruits including pineapple, banana, apple, strawberries, kiwi, and pieces of bread with a small container of dipping sauce in the center.
Description for image 3: Four lemurs sitting on top of a thatched roof.
Description for image 4: An individual is seated on a simple wooden chair operating a vintage sewing machine. The surroundings show a worn, slightly peeling wall with an open wooden door to the side. The floor is tiled, with a few scattered items including a cloth bag and a bottle.
Description for image 5: A young skunk stands on a table near a feeding bottle among various papers and a cloth.
Description for image 6: A small, old-fashioned metal bucket with a wire handle, displayed against a rustic background.
Description for image 7: A trash bin wrapped in a black liner with a NYC 2012 candidate city logo, next to a yellow mop bucket, located in 